In [23]:
from azureml.core import Workspace

ws = Workspace.from_config()

In [24]:
from azureml.core.model import Model

model1 = Model.register(model_path="./pipeline_processing.pkl",
                    model_name="pipeline_processing",
                    workspace=ws)

model2 = Model.register(model_path="./modelML2.pkl",
                    model_name="modelML2",
                    workspace=ws)

Registering model pipeline_processing
Registering model modelML2


In [25]:
import os

folder_name = './'
script_file = os.path.join(folder_name,"score_script.py")

In [26]:
%%writefile $script_file
import json
import pickle
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
    global model_pipe
    global model_xgb
        
    model_path1 = Model.get_model_path('pipeline_processing')
    model_pipe = joblib.load(model_path1)
    model_path2 = Model.get_model_path('modelML2')
    model_xgb = pickle.load(open(model_path2, "rb"))


def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    df = pd.DataFrame(data ,columns=["CreditScore", "Geography", "Gender", "Age", "Tenure", "Balance", "NumOfProducts", "HasCrCard", "IsActiveMember", "EstimatedSalary"])  
    predictions = model_xgb.predict_proba(model_pipe.transform(df))
    return json.dumps(predictions.tolist())

Overwriting ./score_script.py


In [27]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package('scikit-learn=0.22.1')
myenv.add_conda_package('xgboost')

env_file = os.path.join(folder_name,"scoring_env.yml")
with open(env_file,"w") as f:
    f.write(myenv.serialize_to_string())

In [28]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core import Model

# Configure the scoring environment
inference_config = InferenceConfig(runtime= "python",
                                   entry_script=script_file,
                                   conda_file=env_file)

deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

service_name = "first-scoring-service"

service = Model.deploy(ws, service_name, [model1, model2], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy


In [29]:
print(service.state)
print(service.get_logs())

Healthy
2021-02-15T13:50:56,857416900+00:00 - gunicorn/run 
/usr/sbin/nginx: /azureml-envs/azureml_82ee99abb7a87bcb27f9ff905adf82e6/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_82ee99abb7a87bcb27f9ff905adf82e6/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_82ee99abb7a87bcb27f9ff905adf82e6/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_82ee99abb7a87bcb27f9ff905adf82e6/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_82ee99abb7a87bcb27f9ff905adf82e6/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-02-15T13:50:56,859576925+00:00 - nginx/run 
2021-02-15T13:50:56,868849264+00:00 - iot-server/run 
2021-02-15T13:50:56,881331489+00:00 - rsyslog/run 


In [30]:
import json

x_new = [[596, "Germany", "Male", 32, 3, 96709.1, 2, 0, 0, 41788.4]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Call the web service, passing the input data (the web service will also accept the data in binary format)
predictions = service.run(input_data = input_json)

# Get the predicted class - it'll be the first (and only) one.
result = json.loads(predictions)
print(result)

[[0.9785522222518921, 0.021447796374559402]]


In [31]:
endpoint = service.scoring_uri
print(endpoint)

http://8aaf3742-a34f-4abe-8473-be7eb300dda6.francecentral.azurecontainer.io/score


In [32]:
import requests
import json

x_new = [[596, "Germany", "Male", 32, 3, 96709.1, 2, 0, 0, 41788.4]]

# Convert the array to a serializable list in a JSON document
input_json = json.dumps({"data": x_new})

# Set the content type
headers = { 'Content-Type':'application/json' }

predictions = requests.post(endpoint, input_json, headers = headers)
predicted_classes = json.loads(predictions.json())

print(predicted_classes)

[[0.9785522222518921, 0.021447796374559402]]


In [22]:
service = ws.webservices['first-scoring-service']
service.delete()